In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading PL data
PL_dict = zload(data_folder + 'PL_dict.pkz') 

In [5]:
# PL_dict

In [6]:
mu_1 = PL_dict['PL_AM']  # normal PL for AM period
mu_2 = PL_dict['PL_MD']  # normal PL for MD period
mu_3 = PL_dict['PL_PM']  # normal PL for PM period
mu_4 = PL_dict['PL_NT']  # normal PL for NT period

mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

mu_2 = mu_adjust(mu_2)  # normal PL
mu_02, mu2, mu_12, P2, G_12, H_12, U_12 = ChainGen_(mu_2)

mu_3 = mu_adjust(mu_3)  # normal PL
mu_03, mu3, mu_13, P3, G_13, H_13, U_13 = ChainGen_(mu_3)

mu_4 = mu_adjust(mu_4)  # normal PL
mu_04, mu4, mu_14, P4, G_14, H_14, U_14 = ChainGen_(mu_4)

<string>:486: RuntimeWarning: covariance is not positive-semidefinite.


In [7]:
beta = 0.001
G_list = [G_11, G_12, G_13, G_14]
H_list = [H_11, H_12, H_13, H_14]
U_list = [U_11, U_12, U_13, U_14]

eta_wc = {}
eta_Sanov = {}

# Get thresholds for Hoeffding's test corresponding to sample length n  
def etaWC(n):
    return HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)

In [8]:
# Loading test data

with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [9]:
sample_path_jam_test_data_sorted_trans['0']

{u'alphaTrans': 39.0,
 u'startTime': 300.0742,
 u'uuid': [u'4a27354e-e8c6-373c-9863-f6bfe6d7a4d4',
  u'd836e23a-c958-37a7-b126-c70258b61a7a']}

In [10]:
i = 0  # index of windows
sam_gap = 100
win_size = 600
win_list = []
while i * sam_gap + win_size < 24 * 3600:
    win_list.append((i * sam_gap, i * sam_gap + win_size))
    i += 1

In [11]:
i

858

In [12]:
win_num = len(win_list)
jam_num = len(sample_path_jam_test_data_sorted_trans)

win_jam = {}
for i in range(win_num):
    key = str(i)
    value = []
    for j in range(jam_num):
        if ((sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] >= win_list[i][0]) \
            and (sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] < win_list[i][1])):
            value.append(str(j))
    win_jam[key] = value

In [13]:
win_jam['857'], win_list[857][0], win_num 

(['3733',
  '3734',
  '3735',
  '3736',
  '3737',
  '3738',
  '3739',
  '3740',
  '3741',
  '3742',
  '3743',
  '3744',
  '3745',
  '3746',
  '3747',
  '3748',
  '3749',
  '3750',
  '3751',
  '3752',
  '3753',
  '3754'],
 85700,
 858)

In [14]:
sample_path_jam_test_data_sorted_trans[win_jam['100'][0]]['startTime']

10500.0035

In [15]:
KL_list = []  # The list of the KL divergences; each is for a detection window
etaWC_list = []
for i in range(win_num):
    sample_path_jam = [sample_path_jam_test_data_sorted_trans[key]['alphaTrans'] for key in win_jam[str(i)]]
    n = len(sample_path_jam)
    if (n != 0):
        etaWC_list.append(etaWC(n))
        KL_1 = KL_est(sample_path_jam, mu_1)
        KL_2 = KL_est(sample_path_jam, mu_2)
        KL_3 = KL_est(sample_path_jam, mu_3)
        KL_4 = KL_est(sample_path_jam, mu_4)
        KL_list.append(min([KL_1, KL_2, KL_3, KL_4]))
    else:
        etaWC_list.append(1e-6)
        KL_list.append(0)

In [16]:
win_jam[str(100)], win_list[100][1]

(['207', '208', '209', '210', '211', '212', '213', '214'], 10600)

In [17]:
time_range = [win_list[i][0] / 60.0 for i in range(win_num)]
plot_points(time_range, KL_list, etaWC_list)
plt.ylabel('divergence')
plt.xlabel('time (min)')
pylab.ylim(-0.1, max(etaWC_list)+0.5)
pylab.xlim(0, max(time_range) + 10)
plt.savefig(data_folder + 'detec_results_WC_robust.pdf')
plt.show()

In [18]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(858, 858, 858, [0.0, 1.6666666666666667])

In [19]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [20]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)

In [34]:
sample_path_jam_test_data_sorted['3551']

{u'alpha': 0.0,
 u'length': 3568.0,
 u'numPts': 25.0,
 u'speed': 8.919444444444444,
 u'startTime': 82500.3026,
 u'uuid': u'949eca59-4be7-3352-af0e-959032a35a2c'}

In [22]:
len(set(potential_ano_list)), set(potential_ano_list)

(658,
 {'1167',
  '1168',
  '1169',
  '1170',
  '1171',
  '1172',
  '1173',
  '1174',
  '1175',
  '1176',
  '1177',
  '1178',
  '1179',
  '1180',
  '1181',
  '1182',
  '1183',
  '1184',
  '1185',
  '1186',
  '1187',
  '1188',
  '1189',
  '1190',
  '1191',
  '1192',
  '1193',
  '1194',
  '1195',
  '1526',
  '1527',
  '1528',
  '1529',
  '1530',
  '1531',
  '1532',
  '1533',
  '1534',
  '1535',
  '1536',
  '1537',
  '1538',
  '1539',
  '1540',
  '1541',
  '1542',
  '1543',
  '1544',
  '1545',
  '1546',
  '1547',
  '1548',
  '1549',
  '1550',
  '1551',
  '1552',
  '1553',
  '1554',
  '1555',
  '1556',
  '1557',
  '1558',
  '1559',
  '1560',
  '1561',
  '1599',
  '1600',
  '1601',
  '1602',
  '1603',
  '1604',
  '1605',
  '1606',
  '1607',
  '1608',
  '1609',
  '1610',
  '1611',
  '1612',
  '1613',
  '1614',
  '1615',
  '1616',
  '1617',
  '1618',
  '1619',
  '1620',
  '1621',
  '1622',
  '1623',
  '1624',
  '1625',
  '1626',
  '1627',
  '1628',
  '1629',
  '1630',
  '1631',
  '1632',
  '1

In [23]:
len(sample_path_jam_test_data_sorted_trans)

3755

In [25]:
800 / 60.0, 1000 / 60.0, 1200 / 60.0, 1400 / 60.0

(13.333333333333334, 16.666666666666668, 20.0, 23.333333333333332)

In [32]:
# Refine the detection results; if a jam's length is larger than 1 km,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > 1000):
        ano_list.append(key)

In [33]:
len(ano_list), ano_list

(67,
 ['3551',
  '3548',
  '2917',
  '2855',
  '3204',
  '2982',
  '2985',
  '1533',
  '1622',
  '1621',
  '1627',
  '3209',
  '2949',
  '2947',
  '2946',
  '2943',
  '1195',
  '1178',
  '2404',
  '3568',
  '3562',
  '3567',
  '3206',
  '3395',
  '2371',
  '1877',
  '2887',
  '2888',
  '1619',
  '2979',
  '2973',
  '3179',
  '1547',
  '1544',
  '1540',
  '2968',
  '3405',
  '1188',
  '2190',
  '2195',
  '3164',
  '1867',
  '1866',
  '2890',
  '3183',
  '1609',
  '2849',
  '2842',
  '3043',
  '3042',
  '3047',
  '1554',
  '3006',
  '1855',
  '2927',
  '2926',
  '1634',
  '1635',
  '3053',
  '2814',
  '2817',
  '2180',
  '2215',
  '1189',
  '1181',
  '2358',
  '2357'])